In [ ]:
import os, sys, math
import numpy as np
import pandas as pd
from math import cos, sin, asin, sqrt, tan
import matplotlib.pyplot as plt
sys.path.append("putilities")
import utilibs as ul
from scipy.integrate import simps
from numpy import trapz

In [ ]:
flist = ul.get_flist(r"D:\Akademik\P3MI\Unstabilized Approach\Energy Management\em-param", "csv")
len(flist)

In [ ]:
def haversine(lon1_rad, lat1_rad, lon2_rad, lat2_rad):
    """
    Calculate the great circle distance between two points 
    on the earth
    """
    # Haversine formula 
    dlon, dlat = lon2_rad - lon1_rad, lat2_rad - lat1_rad 
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometer is 6371
    dist_km = 6371* c
    return dist_km

def compute_energy(hralt_m, gs_mps):
    """ Calculate actual energy """
    return hralt_m + np.square(gs_mps)/(2*9.80)

def energy_lo(dist_tdown_nm, gs_rad, vref_mps):
    """ Calculate ideal energy when h below IMC gate """
    return dist_tdown_nm * nm_to_m * tan(gs_rad) + vref_mps**2/(2 * 9.80)

def energy_hi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm):
    """ Calculate ideal energy when h above IMC gate """
    pot_energy = dist_tdown_nm * nm_to_m * tan(gs_rad)
    kin_energy = (vref_mps + (dist_cross_nm * 10) * kts_to_mps)**2/(2 * 9.80)
    return pot_energy + kin_energy

def energy_maxlo(dist_tdown_nm, gs_rad, vref_mps):
    """ Calculate upper ideal energy when h below IMC gate """
    return dist_tdown_nm * nm_to_m * tan(1.1063 * gs_rad) + (vref_mps + 20 * kts_to_mps)**2/(2 * 9.80)

def energy_maxhi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm):
    """ Calculate upper ideal energy when h above IMC gate """
    pot_energy = dist_tdown_nm * nm_to_m * tan(1.1063 * gs_rad)
    kin_energy = (vref_mps + 20 * kts_to_mps + (dist_cross_nm * 20) * kts_to_mps)**2/(2 * 9.80)
    return pot_energy + kin_energy

def energy_minlo(dist_tdown_nm, gs_rad, vref_mps):
    """ Calculate lower ideal energy when h below IMC gate """
    return dist_tdown_nm * nm_to_m * tan(0.8937 * gs_rad) + (vref_mps - 5 * kts_to_mps)**2/(2 * 9.80)

def energy_minhi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm):
    """ Calculate lower ideal energy when h above IMC gate """
    pot_energy = dist_tdown_nm * nm_to_m * tan(0.8937 * gs_rad)
    kin_energy = (vref_mps - 5 * kts_to_mps + (dist_cross_nm * 1) * kts_to_mps)**2/(2 * 9.80)
    return pot_energy + kin_energy

def energy_dev(actual_energy, ideal_energy):
    """ Calculate energy deviation """
    return actual_energy - ideal_energy

def energy_upmargin(energy_max, ideal_energy):
    return energy_max - ideal_energy

def energy_downmargin(energy_min, ideal_energy):
    return ideal_energy - energy_min

def energy_factorhi(energy_dev, energy_upmargin):
    """ Calculate energy factor if the energy deviation equal or more than 0 """
    return energy_dev / energy_upmargin

def energy_factorlo(energy_dev, energy_downmargin):
    """ Calculate energy factor if the energy deviation below 0 """
    return energy_dev / energy_downmargin

In [ ]:
kts_to_mps = 0.51444
deg_to_rad = 0.0174533
nm_to_m = 1852
m_to_nm = 1./ nm_to_m
ft_to_m = 0.3048
vref_mps = 143 * kts_to_mps
gs_rad = 3 * deg_to_rad
g_mps2 = 9.80
himc_nm = 1000 * 0.000164579
km_to_nm = 0.539957

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 12))
UA_det = []
excess_energy = []
deficit_energy = []
UA_db = {}

for ifile, file in enumerate(flist):
    print(f"{ifile:3.0f} ... {file} ... calculating")
    df = pd.read_csv(file)
    
    actual_energy = []
    ideal_energy, maxb_energy, minb_energy = [], [], []
    energy_factor = []
    
    flag_cross_himc = 1
    dist_tdown_nm, dist_cross_nm = 0, 0
    
    for i in range(len(df)-1):
        energy_height = compute_energy(df.hralt_m.iloc[i], df.gs_mps.iloc[i])
        actual_energy.append(energy_height)
        
        dist_nm = haversine(df.lon_rad.iloc[i], df.lat_rad.iloc[i], 
                            df.lon_rad.iloc[i+1], df.lat_rad.iloc[i+1]) * km_to_nm
        dist_tdown_nm = dist_tdown_nm + dist_nm
    
        if dist_tdown_nm * tan(gs_rad) < himc_nm and flag_cross_himc:
            e_ideal = energy_lo(dist_tdown_nm, gs_rad, vref_mps)
            e_maxb = energy_maxlo(dist_tdown_nm, gs_rad, vref_mps)
            e_minb = energy_minlo(dist_tdown_nm, gs_rad, vref_mps)
            ideal_energy.append(e_ideal)
            maxb_energy.append(e_maxb)
            minb_energy.append(e_minb)
        
        if dist_tdown_nm * tan(gs_rad) >= himc_nm or flag_cross_himc == 0:
            flag_cross_himc = 0
            dist_cross_nm = dist_cross_nm + dist_nm
            e_ideal = energy_hi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm)
            e_maxb = energy_maxhi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm)
            e_minb = energy_minhi(dist_tdown_nm, gs_rad, vref_mps, dist_cross_nm)
            ideal_energy.append(e_ideal)
            maxb_energy.append(e_maxb)
            minb_energy.append(e_minb)
        
        e_dev = energy_dev(energy_height, e_ideal)
        e_up = energy_upmargin(e_maxb, e_ideal)
        e_down = energy_downmargin(e_minb, e_ideal)
        #print(e_dev, e_up, e_down)
        
        if e_dev < 0:
            e_fac = energy_factorlo(e_dev, e_down)
            energy_factor.append(e_fac)
        
        if e_dev >= 0:
            e_fac = energy_factorhi(e_dev, e_up)
            energy_factor.append(e_fac)
    
    ax[0].plot(df.dist_nm[:-1], actual_energy, lw=0.25, color='#1f77b4')
    ax[0].plot(df.dist_nm[:-1], ideal_energy, lw=0.25, color="green" )
    ax[0].plot(df.dist_nm[:-1], maxb_energy, lw=0.25, color="red" )
    ax[0].plot(df.dist_nm[:-1], minb_energy, lw=0.25, color="red" )
    
    ax[1].plot(df.dist_nm[:-1], energy_factor, lw=0.25, color='#1f77b4')
    ax[1].plot([0, 10], [1, 1], lw=0.25, color="red")
    ax[1].plot([0, 10], [-1, -1], lw=0.25, color="red")
    
    y2_2 = np.empty([len(df)-1, 1])
    y2_3 = np.empty([len(df)-1, 1])
    
    y2_2.fill(1)
    y2_3.fill(-1)
    
    x_upper = []
    y1_upper = []
    y2_upper = []
    x_lower = []
    y1_lower = []
    y2_lower = []
    
    for j in range(len(df)-1):
        if (df.dist_nm.iloc[j] >= 0.5) & (df.dist_nm.iloc[j] <= 10) & (energy_factor[j] > 1):
            x_upper.append(df.dist_nm.iloc[j])
            y1_upper.append(energy_factor[j])
            y2_upper.append(1)
            
        if (df.dist_nm.iloc[j] >= 3) & (df.dist_nm.iloc[j] <= 6) & (energy_factor[j] < -1):
            x_lower.append(df.dist_nm.iloc[j])
            y1_lower.append(energy_factor[j])
            y2_lower.append(-1)
    
    if (len(x_upper) != 0) & (len(y1_upper) != 0) & (len(y2_upper) != 0):
        area1_upper_simps = abs(simps(y1_upper, x_upper))
        area2_upper_simps = abs(simps(y2_upper, x_upper))
        area_upper_simps = abs(area1_upper_simps - area2_upper_simps)
        
        area1_upper_trapz = abs(trapz(y1_upper, x_upper))
        area2_upper_trapz = abs(trapz(y2_upper, x_upper))
        area_upper_trapz = abs(area1_upper_trapz - area2_upper_trapz)
        
        excess = 'Yes'
    else:
        area_upper_simps = 0
        area_upper_trapz = 0
        excess = 'No'
    
        
    if (len(x_lower) != 0) & (len(y1_lower) != 0) & (len(y2_lower) != 0):
        area1_lower_simps = abs(simps(y1_lower, x_lower))
        area2_lower_simps = abs(simps(y2_lower, x_lower))
        area_lower_simps = abs(area1_lower_simps - area2_lower_simps)
    
        area1_lower_trapz = abs(trapz(y1_lower, x_lower))
        area2_lower_trapz = abs(trapz(y2_lower, x_lower))
        area_lower_trapz = abs(area1_lower_trapz - area2_lower_trapz)
        
        deficit = 'Yes'
    else:
        area_lower_simps = 0
        area_lower_trapz = 0
        deficit = 'No'
    
    area_simps = area_upper_simps + area_lower_simps
    area_trapz = area_upper_trapz + area_lower_trapz
    #print(area_simps, area_trapz)
    
    threshold = 5
    unstable_flight = []
    
    if (area_simps < threshold) & (area_trapz < threshold):
        SA = 'Yes'
    else:
        ax[1].fill_between(x_lower, y2_lower, y1_lower, color = 'grey', alpha = 0.5)
        ax[1].fill_between(x_upper, y2_upper, y1_upper, color = 'grey', alpha = 0.5)
        unstable_flight.append(file)
        unstable_flight.append(area_simps)
        unstable_flight.append(area_trapz)
        #print(unstable_flight)
        SA = 'No'
        
    excess_energy.append(excess)
    deficit_energy.append(deficit)
    UA_det.append(SA)
    UA_db[f'{ifile}'] = [file, excess_energy[ifile], deficit_energy[ifile], UA_det[ifile]]


#ax[0].set_xlabel('Distance travelled from TD point, nm', fontsize=18)
ax[1].set_xlabel('Distance travelled from TD point, nm', fontsize=18)

ax[0].set_ylabel('Energy height, m', fontsize=18)
ax[1].set_ylabel('Energy factor', fontsize=18)

for axes in ax:
    axes.tick_params(axis='both', which='major', labelsize=14)
    axes.set_xlim([0, 10])

#plt.savefig('threshold 5 all plot energy height and energy_factor using hralt and gs.png', dpi=300)

In [ ]:
UA_energy = pd.DataFrame.from_dict(UA_db, orient='index', columns=['File_name', 'UA_Excess', 'UA_Deficit', 'SA'])
stable = 0
unstable = 0
excess = 0
deficit = 0
both = 0
th5 = []
#print(UA_energy)
#UA_energy.to_excel("threshold area 5.xlsx")

for i in range(len(UA_energy)):
    if UA_energy.SA.iloc[i] == 'Yes':
        stable += 1
        th5.append(1)
    else:
        unstable += 1
        th5.append(0)
        
    if (UA_energy.UA_Excess.iloc[i] == 'Yes') & (UA_energy.UA_Deficit.iloc[i] == 'No') & (UA_energy.SA.iloc[i] == 'No') :
        print(UA_energy.File_name.iloc[i])
        excess += 1
        
    if (UA_energy.UA_Excess.iloc[i] == 'No') & (UA_energy.UA_Deficit.iloc[i] == 'Yes') & (UA_energy.SA.iloc[i] == 'No'):
        print(UA_energy.File_name.iloc[i])
        deficit += 1
        
    if (UA_energy.UA_Excess.iloc[i] == 'Yes') & (UA_energy.UA_Deficit.iloc[i] == 'Yes') & (UA_energy.SA.iloc[i] == 'No'):
        print(UA_energy.File_name.iloc[i])
        both += 1
        
total = stable + unstable

print("Threshold Area: ", threshold)
print("Stable Approach: ", stable)
print("Unstable Approach: ", unstable)
print("UA Excess: ", excess)
print("UA Deficit: ", deficit)
print("UA Excess and Deficit: ", both)
print("Total Flights: ", total)
print(th5)